# Experimental Protocol

## Introduction
Subjects performed different motor/imagery tasks while 64-channel EEG were recorded using the BCI2000 system ([BCI2000 website](http://www.bci2000.org)). 

## Experimental Runs
Each subject performed 14 experimental runs:
1. Two one-minute baseline runs:
   - Baseline with eyes open
   - Baseline with eyes closed
2. Three two-minute runs of each of the following tasks:
   1. Task 1: Open and close left or right fist
   2. Task 2: Imagine opening and closing left or right fist
   3. Task 3: Open and close both fists or both feet
   4. Task 4: Imagine opening and closing both fists or both feet

## Summary of Experimental Runs
1. Baseline, eyes open
2. Baseline, eyes closed
3. Task 1 (open and close left or right fist)
4. Task 2 (imagine opening and closing left or right fist)
5. Task 3 (open and close both fists or both feet)
6. Task 4 (imagine opening and closing both fists or both feet)
7. Task 1
8. Task 2
9. Task 3
10. Task 4
11. Task 1
12. Task 2
13. Task 3
14. Task 4

## Data Format
The data are provided in EDF+ format, containing:
- 64 EEG signals
- Sampled at 160 samples per second
- Annotation channel

## Annotations
Each annotation includes one of three codes (`T0`, `T1`, or `T2`):
- `T0`: Rest
- `T1`: Onset of motion (real or imagined) of
  - Left fist
  - Both fists
- `T2`: Onset of motion (real or imagined) of
  - Right fist
  - Both feet


# Getting all the paths

In [1]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display

data_path=[]
event_data_path=[]
for i in range(1,15):
    if i<10:
      data_path.append('files/S001/S001R0'+str(i)+'.edf')
      event_data_path.append('files/S001/S001R0'+str(i)+'.edf.event')
    else:
      data_path.append('files/S001/S001R'+str(i)+'.edf')
      event_data_path.append('files/S001/S001R'+str(i)+'.edf.event')


# Importing all the paths

In [2]:
raw=[]
for i in range(0,14):
  raw.append(mne.io.read_raw_edf(data_path[i], preload=True))

Extracting EDF parameters from d:\IIIT\IIIT_Shri_City\SEMESTER 6\BCI\Project\dataset\files\S001\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from d:\IIIT\IIIT_Shri_City\SEMESTER 6\BCI\Project\dataset\files\S001\S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from d:\IIIT\IIIT_Shri_City\SEMESTER 6\BCI\Project\dataset\files\S001\S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from d:\IIIT\IIIT_Shri_City\SEMESTER 6\BCI\Project\dataset\files\S001\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Ex

# Correcting the channel mapping of all the raw data

In [3]:
channel_mapping = {'Fc5.': 'FC5', 'Fc3.': 'FC3', 'Fc1.': 'FC1', 'Fcz.': 'FCz', 'Fc2.': 'FC2', 'Fc4.': 'FC4', 'Fc6.': 'FC6', 'C5..': 'C5', 'C3..': 'C3', 'C1..': 'C1', 'Cz..': 'Cz', 'C2..': 'C2', 'C4..': 'C4', 'C6..': 'C6', 'Cp5.': 'CP5', 'Cp3.': 'CP3', 'Cp1.': 'CP1', 'Cpz.': 'CPz', 'Cp2.': 'CP2', 'Cp4.': 'CP4', 'Cp6.': 'CP6', 'Fp1.': 'Fp1', 'Fpz.': 'Fpz', 'Fp2.': 'Fp2', 'Af7.': 'AF7', 'Af3.': 'AF3', 'Afz.': 'AFz', 'Af4.': 'AF4', 'Af8.': 'AF8', 'F7..': 'F7', 'F5..': 'F5', 'F3..': 'F3', 'F1..': 'F1', 'Fz..': 'Fz', 'F2..': 'F2', 'F4..': 'F4', 'F6..': 'F6', 'F8..': 'F8', 'Ft7.': 'FT7', 'Ft8.': 'FT8', 'T7..': 'T7', 'T8..': 'T8', 'T9..': 'T9', 'T10.': 'T10', 'Tp7.': 'TP7', 'Tp8.': 'TP8', 'P7..': 'P7', 'P5..': 'P5', 'P3..': 'P3', 'P1..': 'P1', 'Pz..': 'Pz', 'P2..': 'P2', 'P4..': 'P4', 'P6..': 'P6', 'P8..': 'P8', 'Po7.': 'PO7', 'Po3.': 'PO3', 'Poz.': 'POz', 'Po4.': 'PO4', 'Po8.': 'PO8', 'O1..': 'O1', 'Oz..': 'Oz', 'O2..': 'O2', 'Iz..': 'Iz'}
for i in range(0,14):
	raw[i].rename_channels(channel_mapping)
	raw[i].set_montage('standard_1005')

In [4]:
raw[0].compute_psd()

Effective window size : 12.800 (s)


Data type,Power Spectrum
Units,eeg: V²/Hz
Data source,Raw
Dims,"channel, freq"
Estimation method,welch
Number of channels,64
Number of frequency bins,1025
Frequency range,0.00 – 80.00 Hz


In [5]:
raw[0].info['ch_names']

['FC5',
 'FC3',
 'FC1',
 'FCz',
 'FC2',
 'FC4',
 'FC6',
 'C5',
 'C3',
 'C1',
 'Cz',
 'C2',
 'C4',
 'C6',
 'CP5',
 'CP3',
 'CP1',
 'CPz',
 'CP2',
 'CP4',
 'CP6',
 'Fp1',
 'Fpz',
 'Fp2',
 'AF7',
 'AF3',
 'AFz',
 'AF4',
 'AF8',
 'F7',
 'F5',
 'F3',
 'F1',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT7',
 'FT8',
 'T7',
 'T8',
 'T9',
 'T10',
 'TP7',
 'TP8',
 'P7',
 'P5',
 'P3',
 'P1',
 'Pz',
 'P2',
 'P4',
 'P6',
 'P8',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'O1',
 'Oz',
 'O2',
 'Iz']

In [6]:
raw_temp = raw[2]
annotations_temp = raw_temp.annotations
print(annotations_temp)
codes_temp = annotations_temp.description
print("Names of 64 Channels")
print(raw_temp.ch_names)
print("Acitivity/Target associated with samples")
print(codes_temp)
annotations=[]
codes=[]
for i in range(0,14):
  annotations.append(raw[i].annotations)
  codes.append(annotations[i].description)


<Annotations | 30 segments: T0 (15), T1 (8), T2 (7)>
Names of 64 Channels
['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FT8', 'T7', 'T8', 'T9', 'T10', 'TP7', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']
Acitivity/Target associated with samples
['T0' 'T2' 'T0' 'T1' 'T0' 'T1' 'T0' 'T2' 'T0' 'T2' 'T0' 'T1' 'T0' 'T1'
 'T0' 'T2' 'T0' 'T1' 'T0' 'T2' 'T0' 'T2' 'T0' 'T1' 'T0' 'T1' 'T0' 'T2'
 'T0' 'T1']


In [33]:
df = pd.DataFrame(raw[2].get_data().T, columns=raw[2].ch_names)
df = df[~(df == 0).all(axis=1)]
timeArray = np.array([round(x,3) for x in np.arange(0,len(df)/160,.00625)])
print(timeArray)
# df = df.transpose()
df

[0.00000e+00 6.00000e-03 1.20000e-02 ... 1.24481e+02 1.24488e+02
 1.24494e+02]


,FC5,FC3,FC1,FCz,FC2,FC4,FC6,C5,C3,C1,...,P8,PO7,PO3,POz,PO4,PO8,O1,Oz,O2,Iz
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19915,0.000023,0.000016,0.000014,0.000010,0.000010,0.000002,0.000017,0.000023,0.000024,0.000024,...,0.000013,0.000058,0.000053,0.000056,0.000042,0.000046,0.000076,0.000068,0.000086,0.000032
19916,0.000023,0.000023,0.000024,0.000020,0.000023,0.000014,0.000022,0.000031,0.000031,0.000038,...,0.000050,0.000061,0.000052,0.000072,0.000072,0.000079,0.000072,0.000082,0.000124,0.000051
19917,0.000023,0.000014,0.000011,0.000003,0.000005,-0.000003,0.000001,0.000021,0.000019,0.000025,...,0.000044,0.000024,0.000024,0.000066,0.000074,0.000085,0.000062,0.000097,0.000156,0.000075
19918,0.000038,0.000027,0.000028,0.000019,0.000019,0.000012,0.000009,0.000034,0.000032,0.000040,...,0.000025,0.000005,0.000011,0.000053,0.000051,0.000043,0.000044,0.000081,0.000140,0.000066


In [37]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def perform_pca(df, n_components=4):
    # Standardize the features by removing the mean and scaling to unit variance
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)
    
    # Initialize PCA
    pca = PCA(n_components=n_components)
    
    # Perform PCA
    pca_result = pca.fit_transform(scaled_data)
    
    # Create a DataFrame to hold the PCA results
    pca_df = pd.DataFrame(data=pca_result, columns=[f"PC{i+1}" for i in range(pca_result.shape[1])])
    
    # Explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_
    
    return pca_df, explained_variance_ratio

# Perform PCA
pca_df, explained_variance_ratio = perform_pca(df,n_components=8)

# Print PCA DataFrame
print("PCA DataFrame:")
print(pca_df)

# Print explained variance ratio
print("\nExplained Variance Ratio:")
print(explained_variance_ratio)


PCA DataFrame:
            PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0     -4.725355  0.831617  1.992440  2.177418  0.061248  0.344767  1.501983   
1     -6.064683  1.895146  2.469918  2.055759 -0.099237 -0.054494  0.211302   
2     -7.585848  2.092894  2.143338  1.690143  0.246902 -0.063763 -0.612204   
3     -8.661872  1.752941  2.477908  1.155384 -0.331436  0.507296  0.060190   
4     -9.854295  2.720544  2.832536  1.118382  0.074556  0.336813 -0.384324   
...         ...       ...       ...       ...       ...       ...       ...   
19915  3.873919 -1.812209 -0.126683 -1.482054 -0.076325  1.363902 -0.472923   
19916  5.809691 -3.151095 -0.476298 -1.648560  0.254963 -0.073765  0.108919   
19917  4.426765 -3.499876 -0.779747 -1.944241 -0.119375 -1.128130 -1.032874   
19918  5.511496 -1.594895 -0.807412 -0.895395  0.116549 -1.027015 -0.398174   
19919  3.195238 -0.154394 -0.234946  0.662164  0.367577 -0.950696 -0.045382   

            PC8  
0      0.249769  
